# Optimization using genetic algorithm

## Import libraries

In [ ]:

import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix


##  Load the dataset

In [ ]:
df = pd.read_csv(r"C:\Users\amnsh\OneDrive\Desktop\DAA\Dataset_cia2\Bank_Personal_Loan_Modelling.csv")

## Preprocess the data

In [ ]:
X = df.drop(["ID", "Personal Loan"], axis=1).values
y = df["Personal Loan"].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Neural network architecture

In [ ]:
model = Sequential()
model.add(Dense(units=6, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(units=6, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

## Fitness function

In [ ]:

def fitness(weights):
    model.set_weights(weights)
    y_pred = model.predict(X_train)
    y_pred = np.round(y_pred)
    return -1 * np.sum(y_pred == y_train) / len(y_train)



## Cultural Algorithm

In [ ]:
class CulturalAlgorithm:
    def __init__(self, population_size, chromosome_length, num_generations, alpha):
        self.population_size = population_size
        self.chromosome_length = chromosome_length
        self.num_generations = num_generations
        self.alpha = alpha
        self.population = self.initialize_population()

    def initialize_population(self):
        population = []
        for i in range(self.population_size):
            chromosome = np.random.uniform(low=-1, high=1, size=self.chromosome_length)
            fitness_value = fitness(chromosome)
            population.append((chromosome, fitness_value))
        return population

    def selection(self, population):
        population = sorted(population, key=lambda x: x[1])
        fittest_individual = population[0]
        least_fit_individual = population[-1]
        new_population = [fittest_individual, least_fit_individual]
        for i in range(self.population_size - 2):
            chromosome = population[i][0]
            fitness_value = population[i][1]
            if np.random.uniform() < np.exp(-self.alpha * fitness_value):
                new_population.append((chromosome, fitness_value))
        return new_population

    def crossover(self, population):
        new_population = []
        for i in range(self.population_size - 2):
            parent1 = population[np.random.randint(len(population))][0]
            parent2 = population[np.random.randint(len(population))][0]
            crossover_point = np.random.randint(self.chromosome_length)
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            fitness_value1 = fitness(child1)
            fitness_value2 = fitness(child2)
            new_population.append((child1, fitness_value1))
            new_population.append((child2, fitness_value2))
        return new_population

    def mutation(self, population):
        new_population = []
        for i in range(self.population_size - 2):
            chromosome = population[np.random.randint(len(population))][0]
            mutation_point = np.random.randint(self.chromosome_length)
            mutation_value = np.random.uniform(low=-1, high=1)
            chromosome[mutation_point] = mutation_value
            fitness_value = fitness(chromosome)
            new_population.append((chromosome, fitness_value))
        return new_population

    def run(self):
        for i in range(self.num_generations):
            print(f"Generation {i+1}/{self.num_generations}")
            population = self.selection(self.population)
            population = self.crossover(population)
            population = self.mutation(population)
            self.population = population
        
        fittest_individual = sorted(self.population, key=lambda x: x[1])[0][0]
        return fittest_individual



In [ ]:
weights = model.get_weights()
chromosome_length = model.count_params()
ca = CulturalAlgorithm(population_size=20, chromosome_length=chromosome_length, num_generations=100, alpha=0.01)

best_weights = ca.run()

model.set_weights(best_weights)
print("Best individuals:[]"{best_weights})

In [ ]:
y_pred = model.predict(X_test)
y_pred = np.round(y_pred)
cm = confusion_matrix(y_test, y_pred)
accuracy = (cm[0][0] + cm[1][1]) / len(y_test)
print(f"Accuracy: {accuracy*100:.2f}%")